In [1]:
# noexport
!typech survey_asknext.ipynb

[NbConvertApp] Converting notebook survey_asknext.ipynb to python
survey_asknext.py:101: error: Name 'day_set' is not defined
survey_asknext.py:102: error: Name 'day_set' is not defined
survey_asknext.py:103: error: Name 'day_set' is not defined
survey_asknext.py:219: error: Name 'survey_choice_to_difficulty_choice_counts' is not defined


In [2]:
from train_utils import *
from plot_utils import *
from scipy.stats import chisquare


In [3]:
def get_most_common_key_in_dict(d):
  most_common = None
  for k in sorted(d.keys()):
    if most_common == None:
      most_common = k
    elif d[k] > d[most_common]:
      most_common = k
  return most_common

def get_most_common_survey_result_for_user(user):
  results = get_survey_results_for_user(user)
  return get_most_common_key_in_dict(results)

def get_most_common_survey_result_for_install(install):
  results = get_survey_results_for_install(install)
  return get_most_common_key_in_dict(results)

In [4]:
def get_survey_results_across_all_installs():
  output = Counter()
  for install in get_installs_with_asknext_survey():
    for k,v in get_survey_results_for_install(install).items():
      output[k] += v
  return output

def get_survey_results_across_all_users():
  output = Counter()
  for user in get_users_with_asknext_survey():
    for k,v in get_survey_results_for_user(user).items():
      output[k] += v
  return output

In [5]:
def get_survey_results_for_install(install):
  output = Counter()
  collection_items = get_collection_for_install(install, 'internal:choose_difficulty')
  for item in collection_items:
    if ('developer_mode' in item) and (item['developer_mode'] == True):
      continue
    if ('unofficial_version' in item):
      continue
    if 'action_type' in item and item['action_type'] == 'asknext_chosen':
      asknext = item['asknext']
      output[asknext] += 1
  return output

def get_survey_results_for_user(user):
  output = Counter()
  collection_items = get_collection_for_user(user, 'internal:choose_difficulty')
  for item in collection_items:
    if ('developer_mode' in item) and (item['developer_mode'] == True):
      continue
    if ('unofficial_version' in item):
      continue
    if 'action_type' in item and item['action_type'] == 'asknext_chosen':
      asknext = item['asknext']
      output[asknext] += 1
  return output

In [6]:
def get_installs_with_asknext_survey():
  output = []
  installs_with_difficulty = get_installs_with_choose_difficulty()
  for install in installs_with_difficulty:
    abtest_settings = get_abtest_settings_for_install(install)
    if abtest_settings.get('frequency_of_choose_difficulty') == 'survey':
      output.append(install)
  return output

def get_users_with_asknext_survey():
  output = []
  users_with_difficulty = get_users_with_choose_difficulty()
  for user in users_with_difficulty:
    abtest_settings = get_abtest_settings(user)
    if abtest_settings.get('frequency_of_choose_difficulty') == 'survey':
      output.append(user)
  return output

In [7]:
# noexport

num_samples = 0
num_users = 0
day_set = set()
for install in get_installs_with_asknext_survey():
  #most_common_result = get_most_common_survey_result_for_install(install)
  results = get_survey_results_for_install(install)
  most_common_result = get_most_common_key_in_dict(results)
  if most_common_result == None:
    continue
  num_choices = sum(results.values())
  num_samples += num_choices
  num_users += 1
  collection_items = get_collection_for_install(install, 'internal:choose_difficulty')
  for item in collection_items:
    if ('developer_mode' in item) and (item['developer_mode'] == True):
      continue
    if ('unofficial_version' in item):
      continue
    if 'action_type' in item and item['action_type'] == 'asknext_chosen':
      #print(item)
      day = item['day']
      day_set.add(day)
  #print(num_choices)
  #break
  #print(results)
  #break

print(num_samples)
print(num_users)

48392
643


In [8]:
# noexpor


print(min(day_set))
print(max(day_set))
print(max(day_set) - min(day_set))

1220
1605
385


In [9]:
def get_survey_choice_to_num_installs_who_choose_it_most_commonly():
  output = Counter()
  for install in get_installs_with_asknext_survey():
    most_common_result = get_most_common_survey_result_for_install(install)
    if most_common_result == None:
      continue
    output[most_common_result] += 1
  return output

def get_survey_choice_to_num_users_who_choose_it_most_commonly():
  output = Counter()
  for user in get_users_with_asknext_survey():
    most_common_result = get_most_common_survey_result_for_user(user)
    if most_common_result == None:
      continue
    output[most_common_result] += 1
  return output

In [10]:
def get_most_common_difficulty_for_user(user):
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  max_count = 0
  max_difficulty = None
  for difficulty,count in difficulty_counts.items():
    if count > max_count:
      max_count = count
      max_difficulty = difficulty
  return max_difficulty

def get_survey_choice_to_difficulty_choice_counts():
  output = {
    'nextvisit': Counter(),
    'hour': Counter(),
    'day': Counter(),
    'week': Counter(),
  }
  for user in get_users_with_asknext_survey():
    collection_items = get_collection_for_user(user, 'internal:choose_difficulty')
    for item in collection_items:
      if ('developer_mode' in item) and (item['developer_mode'] == True):
        continue
      if ('unofficial_version' in item):
        continue
      if 'action_type' in item and item['action_type'] == 'asknext_chosen':
        asknext = item['asknext']
        difficulty = item['difficulty']
        output[asknext][difficulty] += 1
        #output[asknext] += 1
  return output

def plot_survey_choice_to_difficulty_choice_counts():
  plot_dictdict_as_bar(
    get_survey_choice_to_difficulty_choice_counts(),
    ylabel = 'Number of times chosen',
    xlabel = 'Intervention difficulty chosen, along with when to ask about difficulty again',
    title = 'Choices for intervention difficulty and when to ask about difficulty again',
    remap_labels = {'easy': 'Easy', 'medium': 'Medium', 'hard': 'Hard', 'nothing': 'No Intervention', 'nextvisit': 'Next Visit', 'hour': 'Next Hour', 'day': 'Next Day', 'week': 'Next Week'},
    font=dict(size=18),
  )

def plot_survey_choice_counts_raw_per_user():
  survey_results = get_survey_results_across_all_users()
  print(survey_results)
  print(chisquare(list(survey_results.values())))
  plot_dict_as_bar(
    survey_results,
    ylabel = 'Number of times chosen',
    xlabel = 'Choice for when to ask next about intervention difficulty',
    title = 'Choice for when to ask next about intervention difficulty, raw counts'
  )

def plot_survey_choice_counts_raw_per_install():
  survey_results = get_survey_results_across_all_installs()
  print(survey_results)
  print(chisquare(list(survey_results.values())))
  plot_dict_as_bar(
    survey_results,
    ylabel = 'Number of times chosen',
    xlabel = 'Choice for when to ask next about intervention difficulty',
    title = 'Choice for when to ask next about intervention difficulty, raw counts'
  )

def plot_survey_choice_counts_install_normalized():
  survey_choice_to_num_installs_who_choose_it_most_commonly = get_survey_choice_to_num_installs_who_choose_it_most_commonly()
  print(survey_choice_to_num_installs_who_choose_it_most_commonly)
  print(chisquare(list(survey_choice_to_num_installs_who_choose_it_most_commonly.values())))
  plot_dict_as_bar(
    survey_choice_to_num_installs_who_choose_it_most_commonly,
    ylabel = 'Number of users',
    xlabel = 'Choice for when to next ask about intervention difficulty',
    title = 'Users\' most frequent choice for sampling frequency',
    remap_labels = {'nextvisit': 'Next Visit', 'hour': 'Next Hour', 'day': 'Next Day', 'week': 'Next Week'},
    font=dict(size=22),
  )

def plot_survey_choice_counts_user_normalized():
  survey_choice_to_num_users_who_choose_it_most_commonly = get_survey_choice_to_num_users_who_choose_it_most_commonly()
  print(survey_choice_to_num_users_who_choose_it_most_commonly)
  print(chisquare(list(survey_choice_to_num_users_who_choose_it_most_commonly.values())))
  plot_dict_as_bar(
    survey_choice_to_num_users_who_choose_it_most_commonly,
    ylabel = 'Number of users',
    xlabel = 'User\'s most frequent choice for when to next ask about intervention difficulty',
    title = 'Most frequent choice for when to ask next about intervention difficulty, by number of users',
  )

In [11]:
# noexport

print(get_survey_choice_to_difficulty_choice_counts())

{'nextvisit': Counter({'nothing': 22057, 'easy': 535, 'medium': 155, 'hard': 125}), 'hour': Counter({'easy': 2054, 'nothing': 1158, 'medium': 132, 'hard': 36}), 'day': Counter({'nothing': 630, 'medium': 508, 'easy': 464, 'hard': 45}), 'week': Counter({'nothing': 2089, 'easy': 524, 'medium': 507, 'hard': 221})}


In [12]:
# noexport

survey_choice_to_difficulty_choice_counts = get_survey_choice_to_difficulty_choice_counts()

In [13]:
total_count = 0
for survey_choice,difficulty_counts in survey_choice_to_difficulty_choice_counts.items():
  for difficulty,count in difficulty_counts.items():
    total_count += count
print(total_count)

31240


In [14]:
# noexport

plot_survey_choice_to_difficulty_choice_counts()

{'easy': 'Easy', 'medium': 'Medium', 'hard': 'Hard', 'nothing': 'No Intervention', 'nextvisit': 'Next Visit', 'hour': 'Next Hour', 'day': 'Next Day', 'week': 'Next Week'}
Next Visit
Next Hour
Next Day
Next Week


In [15]:
# noexport

plot_survey_choice_counts_raw_per_install()

Counter({'nextvisit': 36942, 'hour': 4939, 'week': 4321, 'day': 2190})
Power_divergenceResult(statistic=68368.80558770045, pvalue=0.0)


In [16]:
# noexport

plot_survey_choice_counts_raw_per_user()

Counter({'nextvisit': 22872, 'hour': 3380, 'week': 3341, 'day': 1647})
Power_divergenceResult(statistic=38981.21306017925, pvalue=0.0)


In [17]:
# noexport

plot_survey_choice_counts_user_normalized()

Counter({'nextvisit': 187, 'week': 159, 'day': 53, 'hour': 45})
Power_divergenceResult(statistic=142.34234234234233, pvalue=1.1813531457717157e-30)


In [18]:
# noexport

plot_survey_choice_counts_install_normalized()

Counter({'nextvisit': 282, 'week': 220, 'day': 74, 'hour': 67})
Power_divergenceResult(statistic=214.78538102643859, pvalue=2.6909002056363194e-46)


In [19]:
# sum(Counter({'nextvisit': 282, 'week': 220, 'day': 74, 'hour': 67}).values())